In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from absbox import API

localAPI = API("https://absbox.org/api/latest",'english')

/opt/homebrew/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


/opt/homebrew/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

✅Connected, local lib:0.19.1, server:0.19.6

In [5]:
myPool = {'assets':[
              ["Mortgage"
              ,{"originBalance": 12000.0
               ,"originRate": ["fix",0.045]
               ,"originTerm": 120
               ,"freq": "monthly"
               ,"type": "level"
               ,"originDate": "2021-02-01"}
              ,{"currentBalance": 10000.0
               ,"currentRate": 0.075
               ,"remainTerm": 80
               ,"status": "current"
               ,"borrowerNum":1000}]],
           'cutoffDate':"2022-03-01"}
  
p = localAPI.runPool(myPool, 
                 assumptions=[{"CPR":0.01}  
                             ,{"CDR":0.01}  
                             ,{"Recovery":(0.7,18)}],  
                 read=True)
p

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum
Date,,,,,,,,,
2022-04-01,9875.71,107.24,37.43,8.52,8.53,0.00,0.00,0.045,988.0
2022-05-01,9751.96,107.46,36.97,8.14,8.15,0.00,0.00,0.045,977.0
2022-06-01,9627.65,107.68,36.50,8.31,8.32,0.00,0.00,0.045,966.0
2022-07-01,9503.87,107.90,36.04,7.94,7.94,0.00,0.00,0.045,955.0
2022-08-01,9379.54,108.13,35.57,8.10,8.10,0.00,0.00,0.045,944.0
...,...,...,...,...,...,...,...,...,...
2030-01-01,0.00,0.00,0.00,0.00,0.00,0.36,0.15,0.000,NaN
2030-02-01,0.00,0.00,0.00,0.00,0.00,0.30,0.12,0.000,NaN
2030-03-01,0.00,0.00,0.00,0.00,0.00,0.22,0.09,0.000,NaN


In [6]:
from absbox.local.generic import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[
        ["Mortgage"
        ,{"originBalance":120,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":120
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"borrowerNum":1000
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":100
             ,"rate":0.07
             ,"originBalance":100
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":20
             ,"rate":0.0
             ,"originBalance":100
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}})
      ,("borrowerFee",{"type":{"numFee":[["DayOfMonth",20],("borrowerNumber",),1]}}))
    ,{"amortizing":[
         #["payFee",["acc01"],['trusteeFee']]
          ["calcFee",'borrowerFee']
         ,["payFee",["acc01"],['borrowerFee']]
         ,["payInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None)

In [8]:
req = localAPI.build_req(test01 ,assumptions=[{"CPR":0.01}  
                             ,{"CDR":0.01}  
                             ,{"Recovery":(0.7,18)}])

In [95]:
r = localAPI.run(test01 ,assumptions=[{"CPR":0.01}  
                             ,{"CDR":0.01}  
                             ,{"Recovery":(0.7,18)}]
                 ,read=True)
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum
Date,,,,,,,,,
2021-06-15,102.07,17.37,1.26,0.28,0.28,0.00,0.00,0.045,862.0
2021-07-31,96.10,5.81,0.38,0.08,0.08,0.00,0.00,0.045,818.0
2021-08-31,90.11,5.83,0.35,0.08,0.08,0.00,0.00,0.045,768.0
2021-09-30,84.13,5.84,0.33,0.07,0.07,0.00,0.00,0.045,721.0
2021-10-31,78.15,5.86,0.31,0.06,0.06,0.00,0.00,0.045,677.0
2021-11-30,72.16,5.87,0.29,0.06,0.06,0.00,0.00,0.045,629.0
2021-12-31,66.18,5.88,0.27,0.05,0.05,0.00,0.00,0.045,578.0
2022-01-31,60.18,5.90,0.24,0.05,0.05,0.00,0.00,0.045,531.0
2022-02-28,54.17,5.91,0.22,0.05,0.05,0.00,0.00,0.045,482.0


In [96]:
r['fees']['borrowerFee']

,balance,payment,due
date,,,
2021-07-26,1705.09,18.91,0
2021-08-20,2516.82,6.27,0
2021-09-20,4046.56,6.26,0
2021-10-20,5482.32,6.24,0
2021-11-20,6830.09,6.23,0
2021-12-20,8081.87,6.22,0
2022-01-20,9231.67,6.20,0
2022-02-20,10287.48,6.19,0
2022-03-20,11245.30,6.18,0


In [62]:
r['accounts']['acc01'][:30]

,balance,change,memo
date,,,
2021-06-15,1.26,1.26,<PoolInflow:CollectedInterest>
2021-06-15,18.63,17.37,<PoolInflow:CollectedPrincipal>
2021-06-15,18.91,0.28,<PoolInflow:CollectedPrepayment>
2021-06-15,18.91,0.00,<PoolInflow:CollectedRecoveries>
2021-07-26,0.00,-18.91,"<SeqPayFee:[""borrowerFee""]>"
2021-07-26,0.00,0.00,"<PayInt:[""A1""],Nothing>"
2021-07-26,0.00,0.00,"<PayPrin:[""A1""],Nothing>"
2021-07-26,0.00,0.00,"<PayPrin:[""B""],Nothing>"
2021-07-26,0.00,0.00,"<PayYield:B,Nothing>"


In [5]:
#Multi Scneario
myAssumption = [{"CPR":0.0}
                ,{"CDR":0.00}]

myAssumption2 = [{"CPR":0.0}
                ,{"CDR":0.01}]

r = localAPI.run(test01
            ,assumptions=myAssumption2
            ,read=True)

# 获取场景 ‘00’ 的结果
r

{'bonds': OrderedDict([('A1',
                           balance  interest  principal  rate  cash  \
               date                                                   
               2021-07-26   100.00      0.00       0.00  0.07  0.00   
               2021-08-20   100.00      0.00       0.00  0.07  0.00   
               2021-09-20   100.00      1.09       0.00  0.07  1.09   
               2021-10-20    95.12      1.32       4.88  0.07  6.20   
               2021-11-20    89.48      0.56       5.64  0.07  6.20   
               2021-12-20    83.80      0.51       5.68  0.07  6.19   
               2022-01-20    78.10      0.49       5.70  0.07  6.19   
               2022-02-20    72.37      0.46       5.73  0.07  6.19   
               2022-03-20    66.57      0.38       5.80  0.07  6.18   
               2022-04-20    60.78      0.39       5.79  0.07  6.18   
               2022-05-20    54.95      0.34       5.83  0.07  6.17   
               2022-06-20    49.10      0.32   

In [6]:
from absbox.local.util import irr
irr(r['bonds']['A1'],init=('2021-06-15',-70))

0.3815125272711019

In [7]:
myPool = {'assets':[
              ["Mortgage"
              ,{"originBalance": 12000.0
               ,"originRate": ["fix",0.045]
               ,"originTerm": 120
               ,"freq": "monthly"
               ,"type": "even"
               ,"originDate": "2021-02-01"}
              ,{"currentBalance": 10000.0
               ,"currentRate": 0.075
               ,"remainTerm": 12
               ,"status": "current"}]
            ,["Mortgage"
              ,{"originBalance": 12000.0
               ,"originRate": ["fix",0.045]
               ,"originTerm": 120
               ,"freq": "monthly"
               ,"type": "level"
               ,"originDate": "2021-02-01"}
              ,{"currentBalance": 10000.0
               ,"currentRate": 0.075
               ,"remainTerm": 12
               ,"status": "current"}]
            ],
           'cutoffDate':"2022-03-01"}
  
localAPI.runPool(myPool, 
                 assumptions=[{"CPR":0.01}  
                             ,{"CDR":0.01}  
                             ,{"Recovery":(0.7,6)}],  
                 read=True)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC
Date,,,,,,,,
2022-04-01,18319.10,1646.80,74.86,17.04,17.06,0.00,0.00,0.045
2022-05-01,16641.74,1647.14,68.57,15.10,15.12,0.00,0.00,0.045
2022-06-01,14965.98,1647.39,62.29,14.18,14.19,0.00,0.00,0.045
2022-07-01,13293.56,1647.73,56.02,12.34,12.35,0.00,0.00,0.045
2022-08-01,11622.91,1647.99,49.76,11.32,11.34,0.00,0.00,0.045
2022-09-01,9954.83,1648.27,43.50,9.90,9.91,0.00,0.00,0.045
2022-10-01,8289.79,1648.63,37.26,8.20,8.21,11.94,5.10,0.045
2022-11-01,6626.76,1648.91,31.02,7.06,7.06,10.57,4.53,0.045
2022-12-01,4966.55,1649.29,24.80,5.46,5.46,9.92,4.25,0.045


In [8]:
myPool = {'assets':[["Mortgage"
              ,{"originBalance": 12000.0
               ,"originRate": ["fix",0.045]
               ,"originTerm": 120
               ,"freq": "monthly"
               ,"type": "level"
               ,"originDate": "2021-02-01"}
              ,{"currentBalance": 10000.0
               ,"currentRate": 0.075
               ,"remainTerm": 12
               ,"status": "current"}]
            ],
           'cutoffDate':"2022-03-01"}
  
localAPI.runPool(myPool, 
                 assumptions=[{"CPR":0.01}  
                             ,{"CDR":0.01}  
                             ,{"Recovery":(0.7,0)}],  
                 read=True)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC
Date,,,,,,,,
2022-04-01,9168.06,814.89,37.43,8.52,8.53,5.97,2.55,0.045
2022-05-01,8336.33,816.60,34.32,7.56,7.57,5.29,2.27,0.045
2022-06-01,7503.85,818.27,31.20,7.10,7.11,4.97,2.13,0.045
2022-07-01,6671.49,819.98,28.09,6.19,6.19,4.33,1.85,0.045
2022-08-01,5838.47,821.65,24.97,5.68,5.69,3.98,1.70,0.045
2022-09-01,5005.19,823.33,21.85,4.97,4.98,3.48,1.49,0.045
2022-10-01,4171.89,825.05,18.73,4.12,4.13,2.89,1.23,0.045
2022-11-01,3338.05,826.74,15.61,3.55,3.55,2.48,1.06,0.045
2022-12-01,2504.08,828.47,12.49,2.75,2.75,1.92,0.82,0.045


In [9]:
myPool = {'assets':[["Loan"
                     ,{"originBalance": 80000
                       ,"originRate": ["floater"
                                       ,0.045
                                       ,{"index":"SOFR3M"
                                         ,"spread":0.015
                                         ,"reset":"quarterly"}]
                       ,"originTerm": 60
                       ,"freq": "semiAnnually"
                       ,"type": "i_p"
                       ,"originDate": "2021-03-01"}
                     ,{"currentBalance": 65000
                       ,"currentRate": 0.06
                       ,"remainTerm": 48
                       ,"status": "current"}]
            ],
           'cutoffDate':"2022-03-01"}
  
asp = [ {"CPR":0.01} ,{"CDR":0.01},{"Recovery":(0.7,0)},{"Rate":["SOFR3M",0.03]} ]
localAPI.runPool(myPool,assumptions=asp, read=True)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC
Date,,,,,,,,
2022-09-01,64477.98,0.00,1482.92,327.50,194.52,136.16,58.35,0.045
2023-03-01,63968.57,0.00,1447.22,319.60,189.81,132.86,56.94,0.045
2023-09-01,63454.84,0.00,1459.39,322.30,191.43,134.00,57.42,0.045
2024-03-01,62950.75,0.00,1432.06,316.26,187.83,131.48,56.34,0.045
2024-09-01,62445.19,0.00,1436.17,317.17,188.39,131.87,56.51,0.045
2025-03-01,61951.83,0.00,1401.59,309.53,183.83,128.68,55.14,0.045
2025-09-01,61454.29,0.00,1413.38,312.14,185.40,129.78,55.62,0.045
2026-03-01,60968.77,0.00,1379.35,304.61,180.91,126.63,54.27,0.045
2026-09-01,60479.13,0.00,1390.95,307.19,182.45,127.71,54.73,0.045


In [10]:
myPool = {'assets':[["Lease"
                     ,{"fixRental": 12.0
                      ,"originTerm": 96
                      ,"freq": ["DayOfMonth",15]
                      ,"originDate": "2022-01-05"}]
                    ,["Lease"
                     ,{"fixRental": 12.0
                      ,"originTerm": 96
                      ,"freq": ["DayOfMonth",15]
                      ,"originDate": "2022-01-05"}]
            ],
           'cutoffDate':"2022-03-01"}
  
asp = []
localAPI.runPool(myPool,assumptions=asp, read=True)

,Rental
Date,
2022-03-15,672
2022-04-15,744
2022-05-15,720
2022-06-15,744
2022-07-15,720
...,...
2029-08-15,744
2029-09-15,744
2029-10-15,720


In [16]:
myPool = {'assets':[["Installment"
                     ,{"originBalance": 1000.0
                      ,"feeRate": ["fix",0.01]
                      ,"originTerm": 12
                      ,"freq": "Monthly"
                      ,"type": "f_p"
                      ,"originDate": "2022-01-01"}
                      ,{"status": "current"
                        ,"currentBalance":1000.0}]
            ],
           'cutoffDate':"2022-01-15"}
  
asp = []
localAPI.runPool(myPool,assumptions=asp, read=True)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC
Date,,,,,,,,
2022-02-01,916.67,83.33,10,0,0,0,0,0.01
2022-03-01,833.34,83.33,10,0,0,0,0,0.01
2022-04-01,750.01,83.33,10,0,0,0,0,0.01
2022-05-01,666.68,83.33,10,0,0,0,0,0.01
2022-06-01,583.35,83.33,10,0,0,0,0,0.01
2022-07-01,500.02,83.33,10,0,0,0,0,0.01
2022-08-01,416.69,83.33,10,0,0,0,0,0.01
2022-09-01,333.36,83.33,10,0,0,0,0,0.01
2022-10-01,250.03,83.33,10,0,0,0,0,0.01


In [17]:
myPool = {'assets':[["Installment"
                     ,{"originBalance": 1000.0
                      ,"feeRate": ["fix",0.01]
                      ,"originTerm": 12
                      ,"freq": "Monthly"
                      ,"type": "f_p"
                      ,"originDate": "2022-01-01"}
                      ,{"status": "current"
                        ,"currentBalance":500.0}]
            ],
           'cutoffDate':"2022-01-15"}
  
asp = []
localAPI.runPool(myPool,assumptions=asp, read=True)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC
Date,,,,,,,,
2022-02-01,458.33,41.66,5,0,0,0,0,0.01
2022-03-01,416.67,41.66,5,0,0,0,0,0.01
2022-04-01,375.00,41.66,5,0,0,0,0,0.01
2022-05-01,333.34,41.66,5,0,0,0,0,0.01
2022-06-01,291.67,41.66,5,0,0,0,0,0.01
2022-07-01,250.01,41.66,5,0,0,0,0,0.01
2022-08-01,208.34,41.66,5,0,0,0,0,0.01
2022-09-01,166.68,41.66,5,0,0,0,0,0.01
2022-10-01,125.01,41.66,5,0,0,0,0,0.01


In [18]:
myPool = {'assets':[["Installment"
                     ,{"originBalance": 1000.0
                      ,"feeRate": ["fix",0.01]
                      ,"originTerm": 12
                      ,"freq": "Monthly"
                      ,"type": "f_p"
                      ,"originDate": "2022-01-01"}
                      ,{"status": "current"
                        ,"currentBalance":416.69}]
            ],
           'cutoffDate':"2022-08-15"}
  
asp = []
localAPI.runPool(myPool,assumptions=asp, read=True)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC
Date,,,,,,,,
2022-09-01,333.36,83.33,10,0,0,0,0,0.01
2022-10-01,250.03,83.33,10,0,0,0,0,0.01
2022-11-01,166.70,83.33,10,0,0,0,0,0.01
2022-12-01,83.37,83.33,10,0,0,0,0,0.01
2023-01-01,0.04,83.33,10,0,0,0,0,0.01


In [19]:
myPool = {'assets':[["Installment"
                     ,{"originBalance": 1000.0
                      ,"feeRate": ["fix",0.01]
                      ,"originTerm": 12
                      ,"freq": "Monthly"
                      ,"type": "f_p"
                      ,"originDate": "2022-01-01"}
                      ,{"status": "current"
                        ,"currentBalance":208.35}]
            ],
           'cutoffDate':"2022-08-15"}
  
asp = []
localAPI.runPool(myPool,assumptions=asp, read=True)

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC
Date,,,,,,,,
2022-09-01,166.68,41.66,5,0,0,0,0,0.01
2022-10-01,125.01,41.66,5,0,0,0,0,0.01
2022-11-01,83.35,41.66,5,0,0,0,0,0.01
2022-12-01,41.68,41.66,5,0,0,0,0,0.01
2023-01-01,0.02,41.66,5,0,0,0,0,0.01


In [1]:
from absbox import API

localAPI = API("https://absbox.org/api/latest",'english')

In [24]:
from absbox.local.generic import Generic

test01 = Generic(
    "split pool income"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),("acc02",{"balance":0}))
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"PAC":
                          [["2021-07-20",800]
                           ,["2021-08-20",710]
                           ,["2021-09-20",630]
                           ,["2021-10-20",0]
                          ]}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["transferBy", {"balPct": 0.1}, "acc01", "acc02"]
         ,["payFee",["acc01"],['trusteeFee']]
         ,["payInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None)

In [25]:
r = localAPI.run(test01,
           assumptions=[],
           read=True)

In [26]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-07-26,800.00,7.86,200.00,0.07,207.86,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-08-20,710.00,3.83,90.00,0.07,93.83,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-09-20,630.00,4.22,80.00,0.07,84.22,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-10-20,527.54,3.62,102.46,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-11-20,424.59,3.13,102.95,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-12-20,320.95,2.44,103.64,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2022-01-20,216.77,1.90,104.18,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2022-02-20,111.97,1.28,104.80,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2022-03-20,6.49,0.60,105.48,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."


In [11]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-07-26,613.56,7.86,386.44,0.07,394.30,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-08-20,510.42,2.94,103.14,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-09-20,407.37,3.03,103.05,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-10-20,303.63,2.34,103.74,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-11-20,199.35,1.80,104.28,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-12-20,94.41,1.14,104.94,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2022-01-20,0.00,0.56,94.41,0.07,94.97,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."


In [14]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-07-26,1000.00,7.86,0.00,0.07,7.86,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-08-20,1000.00,4.79,0.00,0.07,4.79,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-09-20,899.86,5.94,100.14,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-10-20,798.95,5.17,100.91,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-11-20,697.61,4.74,101.34,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-12-20,595.54,4.01,102.07,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2022-01-20,493.00,3.54,102.54,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2022-02-20,389.85,2.93,103.15,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2022-03-20,285.86,2.09,103.99,0.07,106.08,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."


In [8]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum
Date,,,,,,,,,
2021-06-15,1782.95,417.05,54.39,0,0,0,0,0.045,None
2021-07-31,1676.97,105.98,11.88,0,0,0,0,0.045,None
2021-08-31,1570.66,106.31,11.55,0,0,0,0,0.045,None
2021-09-30,1463.61,107.05,10.81,0,0,0,0,0.045,None
2021-10-31,1355.50,108.11,9.75,0,0,0,0,0.045,None
2021-11-30,1246.97,108.53,9.33,0,0,0,0,0.045,None
2021-12-31,1137.42,109.55,8.31,0,0,0,0,0.045,None
2022-01-31,1027.39,110.03,7.83,0,0,0,0,0.045,None
2022-02-28,916.60,110.79,7.07,0,0,0,0,0.045,None


In [9]:
r['accounts']['acc02']

,balance,change,memo
date,,,
2021-07-26,47.14,47.14,"<Transfer:acc01,acc02>"
2021-08-20,58.92,11.78,"<Transfer:acc01,acc02>"
2021-09-20,70.70,11.78,"<Transfer:acc01,acc02>"
2021-10-20,82.48,11.78,"<Transfer:acc01,acc02>"
2021-11-20,94.26,11.78,"<Transfer:acc01,acc02>"
...,...,...,...
2029-08-20,235.62,0.00,"<Transfer:acc01,acc02>"
2029-09-20,235.62,0.00,"<Transfer:acc01,acc02>"
2029-10-20,235.62,0.00,"<Transfer:acc01,acc02>"


In [10]:
r['accounts']['acc01']

,balance,change,memo
date,,,
2021-06-15,54.39,54.39,<PoolInflow:CollectedInterest>
2021-06-15,471.44,417.05,<PoolInflow:CollectedPrincipal>
2021-06-15,471.44,0.00,<PoolInflow:CollectedPrepayment>
2021-06-15,471.44,0.00,<PoolInflow:CollectedRecoveries>
2021-07-26,424.30,-47.14,"<Transfer:acc01,acc02>"
...,...,...,...
2029-12-20,0.00,0.00,"<SeqPayFee:[""trusteeFee""]>"
2029-12-20,0.00,0.00,"<PayInt:[""A1""],Nothing>"
2029-12-20,0.00,0.00,"<PayPrin:[""A1""],Nothing>"


### PayPrincipalViaFormual

In [96]:
from absbox.local.generic import Generic

fm = {"formula": 
        ("substract",("poolBalance",)
                    ,("factor",("poolBalance",), 0.99))}

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[
        ["Mortgage"
        ,{"originBalance":120,"originRate":["fix",0.045],"originTerm":70
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":200
          ,"currentRate":0.08
          ,"remainTerm":60
          ,"borrowerNum":1
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":100
             ,"rate":0.07
             ,"originBalance":100
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":20
             ,"rate":0.0
             ,"originBalance":100
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}})
      ,("borrowerFee",{"type":{"numFee":[["DayOfMonth",20],("borrowerNumber",),1]}}))
    ,{"amortizing":[
          ["calcFee",'borrowerFee']
         ,["payFee",["acc01"],['borrowerFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01","A1",fm]
         ,["payPrin","acc01",["B"]]
         ,["payResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None)

from absbox import API

localAPI = API("https://absbox.org/api/latest",'english')

r = localAPI.run(test01,
           assumptions=[{"Inspect":
                         [(["DayOfMonth",20],fm['formula'])]}],
           read=True)

/opt/homebrew/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

✅Connected, local lib:0.19.1, server:0.19.7

In [97]:
h = "<Substract:{'tag': 'CurrentPoolBalance'},{'tag': 'Factor', 'contents': [{'tag': 'CurrentPoolBalance'}, {'numerator': 19, 'denominator': 20}]}>"
#r['result']['inspect'].items()[1]
r['result']['inspect'][list(r['result']['inspect'].keys())[0]]

,"<Substract:{'tag': 'CurrentPoolBalance'},{'tag': 'Factor', 'contents': [{'tag': 'CurrentPoolBalance'}, {'numerator': 99, 'denominator': 100}]}>"
Date,
2021-03-01,0.00
2021-03-20,0.00
2021-04-20,0.00
2021-05-20,0.00
2021-06-20,0.00
2021-07-20,0.00
2021-08-20,0.00
2021-09-20,0.00
2021-10-20,0.00


In [74]:
r['accounts']['acc01'].loc["2022-04-20"]

,balance,change,memo
date,,,
2022-04-20,2.72,-1.00,"<SeqPayFee:[""borrowerFee""]>"
2022-04-20,2.13,-0.59,"<PayInt:[""A1""]>"
2022-04-20,0.00,-2.13,"[<PayPrin:[""A1""]>, <DS:Substract [CurrentPoolB..."
2022-04-20,0.00,0.00,"<PayPrin:[""B""]>"
2022-04-20,0.00,0.00,"<PayYield:""B"">"


In [95]:
r['accounts']['acc01'].loc["2022-04-20"]

,balance,change,memo
date,,,
2022-04-20,2.72,-1.00,"<SeqPayFee:[""borrowerFee""]>"
2022-04-20,2.13,-0.59,"<PayInt:[""A1""]>"
2022-04-20,0.21,-1.92,"[<PayPrin:[""A1""]>, <DS:Substract [CurrentPoolB..."
2022-04-20,0.00,-0.21,"<PayPrin:[""B""]>"
2022-04-20,0.00,0.00,"<PayYield:""B"">"


In [75]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-07-26,100.00,0.00,0.00,0.07,0.00,"[<PayInt:[""A1""]>, <PayPrin:[""A1""]>]"
2021-08-20,100.00,0.00,0.00,0.07,0.00,"[<PayInt:[""A1""]>, <PayPrin:[""A1""]>]"
2021-09-20,100.00,0.00,0.00,0.07,0.00,"[<PayInt:[""A1""]>, <PayPrin:[""A1""]>]"
2021-10-20,100.00,0.00,0.00,0.07,0.00,"[<PayInt:[""A1""]>, <PayPrin:[""A1""]>]"
2021-11-20,100.00,0.00,0.00,0.07,0.00,"[<PayInt:[""A1""]>, <PayPrin:[""A1""]>]"
2021-12-20,100.00,0.00,0.00,0.07,0.00,"[<PayInt:[""A1""]>, <PayPrin:[""A1""]>]"
2022-01-20,100.00,0.00,0.00,0.07,0.00,"[<PayInt:[""A1""]>, <PayPrin:[""A1""]>]"
2022-02-20,100.00,2.72,0.00,0.07,2.72,"[<PayInt:[""A1""]>, <PayPrin:[""A1""]>]"
2022-03-20,99.95,2.67,0.05,0.07,2.72,"[<PayInt:[""A1""]>, <PayPrin:[""A1""]>]"
